# w266 Fit Analysis
Experimenting to see the impact of less data for training

### Libraries & Packages

In [ ]:
# data processessing packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## NN packages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from sklearn.metrics import classification_report

# NLP packages
!pip install -q transformers
!pip install pydot
from transformers import BertTokenizer,TFAutoModel, TFBertModel, BertForSequenceClassification,TFAutoModelForSequenceClassification




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.3 MB/s eta 0:00:00


### Importing/Preprocessing data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/w266/finalProject/data/first_half_train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/w266/finalProject/data/clean_test_data.csv')

In [ ]:
train_data

,Unnamed: 0,text,label
0,0,Here are Thursday's biggest analyst calls: App...,0
1,1,Buy Las Vegas Sands as travel to Singapore bui...,0
2,2,"Piper Sandler downgrades DocuSign to sell, cit...",0
3,3,"Analysts react to Tesla's latest earnings, bre...",0
4,4,Netflix and its peers are set for a ‘return to...,0
...,...,...,...
16985,16985,KfW credit line for Uniper could be raised to ...,3
16986,16986,KfW credit line for Uniper could be raised to ...,3
16987,16987,Russian sells 1 bln roubles at one-year repo...,3
16988,16988,Global ESG bond issuance posts H1 dip as supra...,3


In [ ]:
# creating train/val datasets
x_train, x_val, y_train, y_val = train_test_split(train_data.text, train_data.label, test_size=0.20, random_state=42)
# creating test datasets
x_test = test_data.text
y_test = test_data.label

In [ ]:
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}\n")
print(f"x_val shape: {x_val.shape}")
print(f"y_val shape: {y_val.shape}\n")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")


x_train shape: (5436,)
y_train shape: (5436,)

x_val shape: (1360,)
y_val shape: (1360,)

x_test shape: (4117,)
y_test shape: (4117,)


### Gloabl Variables

In [ ]:
# Creating of learning rate schedule
num_epochs = 5
num_train_steps = len(x_train) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5,
    end_learning_rate=0.,
    decay_steps=num_train_steps
)

In [ ]:
target_names = ["Analyst Update","Fed | Central Banks",
        "Company | Product News","Treasuries | Corporate Debt",
        "Dividend","Earnings","Energy | Oil",
        "Financials","Currencies","General News | Opinion",
        "Gold | Metals | Materials","IPO","Legal | Regulation",
        "M&A | Investments","Macro","Markets","Politics",
        "Personnel Change","Stock Commentary", "Stock Movement"]

### Utility functions

In [ ]:
# Function for creating tokenized data and outputs for models

def create_datasets(tokenizer, train=x_train, val=x_val, test=x_test):
  # Variables
  # train/val/test = datasets to encode
  # tokenizer = bert tokenizer

  train_encodings = tokenizer(list(train), padding=True, return_tensors='tf')
  valid_encodings = tokenizer(list(val), padding=True, return_tensors='tf')
  test_encodings = tokenizer(list(test), padding=True, return_tensors='tf')

  return train_encodings, valid_encodings, test_encodings


In [ ]:
 # Function for creating model
def create_bert_multiclass_model(model,
                                 num_classes = 20,
                                 hidden_size = 201,
                                 dropout=0.3,
                                 learning_rate=lr_scheduler,
                                 activation='softmax'):
    """
    Build a simple classification model with BERT. Use the Pooler Output for classification purposes.
    """

    bert_model = model

    # building bert inputs
    input_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(None,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    # building bert model
    bert_out = bert_model(bert_inputs)
    pooler_output = bert_out[1] # bert_out.pooler_output

    # building hidden layers
    last_hidden_output = tf.keras.layers.Dense(hidden_size, activation='relu', name='last_hidden_output')(pooler_output)
    last_hidden_output = tf.keras.layers.Dropout(dropout, name='dropout')(last_hidden_output)
    bert_cls_prediction = keras.layers.Dense(num_classes, activation=activation, name='cls_output')(last_hidden_output)

    # compiling model
    bert_cls_model = keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=bert_cls_prediction)
    bert_cls_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                           metrics='accuracy')

    ### END YOUR CODE
    return bert_cls_model

In [ ]:
# Function to show model parameters/shape
def depict_model(model):

  display(model.summary())
  display(keras.utils.plot_model(model, show_shapes=False, show_dtype=False, show_layer_names=True, dpi=90))

### BERT-base model

In [ ]:
# Loading bert-base-uncased tokenizer/model
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
# Creating base-base-uncased encodings
base_train_encodings, base_valid_encodings, base_test_encodings = create_datasets(bert_tokenizer)

In [ ]:
# Creating bert-base-uncased model
bert_base_Fit = create_bert_multiclass_model(bert_model)

In [ ]:
# Running BERT-base-Fit model
bertbase_model_history = bert_base_Fit.fit([base_train_encodings.input_ids, base_train_encodings.token_type_ids, base_train_encodings.attention_mask],
                                                  y_train,
                                                  validation_data=([base_valid_encodings.input_ids, base_valid_encodings.token_type_ids, base_valid_encodings.attention_mask],
                                                  y_val),
                                                  batch_size=8,
                                                  epochs=num_epochs)

Epoch 1/5
680/680 [==============================] - 246s 278ms/step - loss: 1.3053 - accuracy: 0.6391 - val_loss: 0.7068 - val_accuracy: 0.7949
Epoch 2/5
680/680 [==============================] - 167s 245ms/step - loss: 0.5048 - accuracy: 0.8591 - val_loss: 0.6629 - val_accuracy: 0.8404
Epoch 3/5
680/680 [==============================] - 172s 254ms/step - loss: 0.2801 - accuracy: 0.9224 - val_loss: 0.6875 - val_accuracy: 0.8456
Epoch 4/5
680/680 [==============================] - 172s 253ms/step - loss: 0.1524 - accuracy: 0.9582 - val_loss: 0.7738 - val_accuracy: 0.8346
Epoch 5/5
680/680 [==============================] - 165s 242ms/step - loss: 0.1148 - accuracy: 0.9689 - val_loss: 0.7285 - val_accuracy: 0.8493


In [ ]:
bert_base_fit_results = bert_base_Fit.evaluate([base_test_encodings.input_ids, base_test_encodings.token_type_ids, base_test_encodings.attention_mask],
                                         y_test,
                                         batch_size=8)

print(f"Model accuracy: {bert_base_fit_results[1]}\n"+
      f"Model loss: {bert_base_fit_results[0]}")

515/515 [==============================] - 53s 97ms/step - loss: 0.7358 - accuracy: 0.8441
Model accuracy: 0.8440611958503723
Model loss: 0.7357674241065979


In [ ]:
# Computing BERT-base-Fit F1 metric
bert_fit_y_pred = bert_base_Fit.predict([base_test_encodings.input_ids, base_test_encodings.token_type_ids, base_test_encodings.attention_mask])
pred_bert_fit_model = tf.argmax(bert_fit_y_pred, axis=-1)

print(classification_report(y_test, pred_bert_fit_model.numpy(), target_names=target_names, digits=4))



129/129 [==============================] - 47s 324ms/step
                             precision    recall  f1-score   support

             Analyst Update     0.8409    0.5068    0.6325        73
        Fed | Central Banks     0.8984    0.7850    0.8379       214
     Company | Product News     0.8449    0.9014    0.8722       852
Treasuries | Corporate Debt     0.7317    0.7792    0.7547        77
                   Dividend     0.9600    0.9897    0.9746        97
                   Earnings     0.9378    0.9339    0.9358       242
               Energy | Oil     0.8158    0.8493    0.8322       146
                 Financials     0.8434    0.8750    0.8589       160
                 Currencies     0.6522    0.9375    0.7692        32
     General News | Opinion     0.7404    0.7470    0.7437       336
  Gold | Metals | Materials     0.4286    0.2308    0.3000        13
                        IPO     0.8125    0.9286    0.8667        14
         Legal | Regulation     0.8667    0.